# Integration using FFTs

Suppose we want to integrate a $2\pi$-periodic function $v(x)$ between two points.

The function can be expanded in Fourier modes as:
\begin{equation}
  v(x) = \sum_{k \in \mathbb{Z}} \hat{v}_k e^{ikx}
\end{equation}

Then, its integral is:
\begin{equation}
  I(a, b) \equiv
  \int_a^b v(x) \, dx =
  \sum_k \hat{v}_k \int_a^b e^{ikx} \, dx
\end{equation}

Note that $\int e^{ikx} \, dx = x$ if $k = 0$, and $e^{ikx} / ik$ otherwise.
Then:
\begin{equation}
  I(a, b) = (b - a) \, \hat{v}_0 +
  \sum_{k \ne 0} \frac{\hat{v}_k}{i k}
  \left( e^{ikb} - e^{ika} \right)
\end{equation}

We can now define $\hat{w}_k$ such that $\hat{w}_k = \hat{v}_k / ik$ for $k \ne 0$
and $\hat{w}_0 = 0$.
Then, it is clear that
\begin{align}
  I(a, b) &= (b - a) \, \hat{v}_0 +
  \sum_k \hat{w}_k
  \left( e^{ikb} - e^{ika} \right)
  \\
  &=
  (b - a) \, \hat{v}_0 + w(b) - w(a),
\end{align}
where
\begin{equation}
  w(x) = \sum_{k \in \mathbb{Z}} \hat{w}_k e^{ikx}
\end{equation}
is the inverse Fourier transform of the $\hat{w}_k$ coefficients.

# Example in Julia

In [1]:
using FFTW

N = 32
L = 2pi

x = LinRange(0, L, N + 1)[1:N]

v  = @. 0.2  + cos(x) - 8 * sin(4x)
vI = @. 0.2x + sin(x) + 2 * cos(4x)  # integral

# Wave numbers
k = rfftfreq(N, 2pi * N / L)
@show k

# Perform real-to-complex FFT
vf = rfft(v)

# Keep mode k = 0 (mean value of v)
# NOTE: it needs to be normalised by the length of the transformed data!!
U = Real(vf[1]) / N

# Set zero mode to 0, and divide the rest by ik.
vf[1] = 0
vf[2:end] ./= im * k[2:end]

# Transform back
w = irfft(vf, N)

# Verify integral
a = searchsortedlast(x, 0)
b = searchsortedlast(x, 0.8 * pi)
@show x[[a, b]] ./ pi

int_analytical = vI[b] - vI[a]
int_fft = U * (x[b] - x[a]) + w[b] - w[a]

println()
println("Integral (analytical): ", int_analytical)
println("Integral (FFT)       : ", int_fft)

k = [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0]
x[[a, b]] ./ pi = [0.0, 0.75]

Integral (analytical): -2.8216543207749836
Integral (FFT)       : -2.821654320774983
